<a href="https://colab.research.google.com/github/paulgureghian/MXNet_Projects/blob/master/MXNet_Gluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Created by Paul A. Gureghian in April 2019.**

### **This scientific notebook will implement a 'Resnet' CNN model using the MXNet Gluon library**

### **And 'Comet.ml' will be used for tracking the model experiments.**

In [1]:
### Install MXNet
!pip install -q mxnet-cu100
print("MXNet installed")

    100% |████████████████████████████████| 487.9MB 36kB/s 
    100% |████████████████████████████████| 13.8MB 2.6MB/s 
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.51 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
MXNet installed


In [2]:
### Install Gluon
!pip install -q gluoncv --upgrade
print("Gluon installed") 

    100% |████████████████████████████████| 348kB 27.5MB/s 
Gluon installed


In [3]:
### Install Comet_ML 
!pip install -q comet_ml
print("Comet installed")

    100% |████████████████████████████████| 102kB 11.2MB/s 
    100% |████████████████████████████████| 389kB 24.0MB/s 
    100% |████████████████████████████████| 204kB 27.5MB/s 
Comet installed


In [4]:
### Import packages
from comet_ml import Experiment

import time
import logging
import argparse

import numpy as np
import mxnet as mx

from mxnet.gluon import nn
from mxnet import gluon, nd
from mxnet import autograd as ag
from mxnet.gluon.data.vision import transforms

from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, TrainingHistory
from gluoncv.data import transforms as gcv_transforms

import itertools
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from sklearn.metrics import confusion_matrix

experiment = Experiment(api_key='NF5R5m1y3wNyy8HjXbsqfx8ZD', project_name='mxnet_comet_demo' , workspace='paulgureghian')

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/paulgureghian/mxnet-comet-demo/d49216b4b5b246d0a3021015886288a6



In [19]:
### Instantiate an 'argparse' object
parser = argparse.ArgumentParser()

print(parser)

opt = parser.parse_args()

print(opt)

ArgumentParser(prog='ipykernel_launcher.py', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)


usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-bb959e8a-b26a-4111-baeb-442e286edb23.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
### Set the number of GPUs to use 
batch_size = 32
classes = 10 
class_labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

num_gpus = 1
batch_size *= max(1, num_gpus)
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
num_workers = 1

In [0]:
### Setup data augmentation
transform_train = transforms.Compose([gcv_transforms.RandomCrop(32, pad=4),
                                      transforms.RandomFlipLeftRight(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4914, 0.4822, 0.4465],
                                                           [0.2023, 0.1994, 0.2010])]) 

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.4914, 0.4822, 0.4465],
                                                          [0.2023, 0.1994, 0.2010])])

In [0]:
### Set a Learning Rate (lr)
lr_decay = 0.001 
lr_decay_epoch = [int(i) for i in lr_decay_epoch.split(',')] + [np.inf]

In [0]:
!pip freeze 